In [3]:
import pandas as pd
import os

csv_path = r'C:\Users\ADMIN\Downloads\ampc\ampc'  


file1 = pd.read_csv(os.path.join(csv_path, 'w1.csv'))  
file2 = pd.read_csv(os.path.join(csv_path, 'w2.csv'))
file3 = pd.read_csv(os.path.join(csv_path, 'w3.csv'))
file4 = pd.read_csv(os.path.join(csv_path, 'w4.csv'))


combined_data = pd.concat([file1, file2, file3, file4], ignore_index=True)


combined_data.to_csv('combined_data.csv', index=False)


shuffled_data = combined_data.sample(frac=1, random_state=42).reset_index(drop=True)

shuffled_data.to_csv('all_data.csv', index=False)



In [9]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score
from sklearn.metrics import accuracy_score
import pandas as pd

data=pd.read_csv('all_data.csv')

# Separate features and class
X = data.drop(columns=['class'])
y = data['class']

# Step 1: Train-Test Split (70% training, 30% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

# Initialize the SVM model
clf = svm.SVC()

# Train the model
clf.fit(X_train, y_train)

# Make predictions
y_pred = clf.predict(X_test)

# Calculate accuracy
accuracy_train_test = accuracy_score(y_test, y_pred)

#Perforn cross validation
cv_scores = cross_val_score(clf,X,y,cv=10)
cv_mean_accuracy = cv_scores.mean()

accuracy_train_test, cv_mean_accuracy

(0.8882201203783319, 0.8918210367572736)

In [4]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score
from sklearn.metrics import accuracy_score
import pandas as pd

data=pd.read_csv('all_data.csv')

# Separate features and class
X = data.drop(columns=['class'])
y = data['class']

# Step 1: Train-Test Split (70% training, 30% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

# Initialize the SVM model
clf = svm.SVC()

# Train the model
clf.fit(X_train, y_train)

# Make predictions
y_pred = clf.predict(X_test)

# Calculate accuracy
accuracy_train_test = accuracy_score(y_test, y_pred)

# Define the parameter grid for hyperparameter tuning
param_grid = {
    'C': [0.1, 1, 10, 100],    # Regularization parameter
    'gamma': [1, 0.1, 0.01, 0.001],  # Kernel coefficient
    'kernel': ['rbf']          # Use the RBF kernel
}

# Set up the GridSearchCV with SVM
grid = GridSearchCV(svm.SVC(), param_grid, refit=True, verbose=2, cv=5)

# Fit the model to the entire dataset to find optimal parameters
grid.fit(X, y)

# Get the best parameters from GridSearchCV
best_params = grid.best_params_

# Use the best parameters to update the SVM model
svm_rbf_best = svm.SVC(**best_params)

# Fit the updated model on the train-test split
svm_rbf_best.fit(X_train, y_train)
y_pred_rbf_best = svm_rbf_best.predict(X_test)
accuracy_test_rbf_best = accuracy_score(y_test, y_pred_rbf_best)

# Perform 10-fold cross-validation with the best model
scores_rbf_best = cross_val_score(svm_rbf_best, X, y, cv=10)
accuracy_cross_val_rbf_best = scores_rbf_best.mean()

print(f"Best Parameters: {best_params}")
print(f"Test Set Accuracy: {accuracy_test_rbf_best}")
print(f"Cross-Validation Accuracy: {accuracy_cross_val_rbf_best}")


Fitting 5 folds for each of 16 candidates, totalling 80 fits
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=  58.1s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time= 1.0min
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time= 1.0min
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=  58.3s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time= 1.0min
[CV] END .......................C=0.1, gamma=0.1, kernel=rbf; total time=  59.3s
[CV] END .......................C=0.1, gamma=0.1, kernel=rbf; total time= 1.0min
[CV] END .......................C=0.1, gamma=0.1, kernel=rbf; total time= 1.0min
[CV] END .......................C=0.1, gamma=0.1, kernel=rbf; total time=  57.2s
[CV] END .......................C=0.1, gamma=0.1, kernel=rbf; total time=  59.5s
[CV] END ......................C=0.1, gamma=0.01, kernel=rbf; total time=  57.1s
[CV] END ......................C=0.1, gamma=0.01

In [5]:
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import accuracy_score
from sklearn import svm

# Assuming X and y are your features and target variable.
data=pd.read_csv('all_data.csv')

X = data.drop(columns=['class'])
y = data['class']


# Step 1: Perform PCA to reduce to 10 principal components
pca = PCA(n_components=10)
X_pca = pca.fit_transform(X)

# Step 2a: Train/Test Split (70% train, 30% test) with hyperparameter tuning using PCA features
param_grid = {
    'C': [0.1, 1, 10],         # Regularization parameter
    'gamma': ['scale', 'auto'], # Kernel coefficient
    'kernel': ['rbf']           # RBF kernel
}

# Initialize SVM with GridSearchCV
clf = svm.SVC()
grid_search_pca = GridSearchCV(clf, param_grid, cv=5, scoring='accuracy')

# Split the PCA-transformed data into training and test sets
X_train_pca, X_test_pca, y_train_pca, y_test_pca = train_test_split(X_pca, y, test_size=0.3, random_state=1)

# Train the SVM model on PCA-transformed data
grid_search_pca.fit(X_train_pca, y_train_pca)

# Get the best model for PCA-transformed data
best_model_train_test_pca = grid_search_pca.best_estimator_

# Predict on the test set with PCA features
y_pred_train_test_pca = best_model_train_test_pca.predict(X_test_pca)

# Calculate accuracy for the train/test split with PCA and hyperparameter tuning
accuracy_train_test_pca = accuracy_score(y_test_pca, y_pred_train_test_pca)
print("Train/Test Split Accuracy with PCA:", accuracy_train_test_pca)

# Step 2b: 10-fold cross-validation with hyperparameter tuning on PCA-transformed data
cross_val_scores_pca = cross_val_score(best_model_train_test_pca, X_pca, y, cv=10)
accuracy_cross_validation_pca = cross_val_scores_pca.mean()
print("10-fold Cross-Validation Accuracy with PCA:", accuracy_cross_validation_pca)


Train/Test Split Accuracy with PCA: 0.8959587274290628
10-fold Cross-Validation Accuracy with PCA: 0.8998186333344679


In [ ]:
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import GridSearchCV

# Select the best 100 features using SelectKBest
selector = SelectKBest(f_classif, k=100)
X_new = selector.fit_transform(X, y)

# Hyperparameters from activity 3 (assuming example hyperparameters: C and gamma)
# These values would be fine-tuned based on prior results from activity 3
# Let's assume the values here for demonstration
param_grid = {'C': [1, 10, 100], 'gamma': [0.001, 0.01, 0.1]}

# Split the data for training and testing (70/30)
X_train_new, X_test_new, y_train_new, y_test_new = train_test_split(X_new, y, test_size=0.3, random_state=1)

# a) Train an SVM with hyperparameter tuning using GridSearchCV
clf_grid = GridSearchCV(svm.SVC(), param_grid, cv=3)  # Using 3-fold CV for tuning hyperparameters
clf_grid.fit(X_train_new, y_train_new)

# Predict and measure accuracy for the best model
y_pred_grid = clf_grid.predict(X_test_new)
accuracy_grid = accuracy_score(y_test_new, y_pred_grid)

# b) Perform 10-fold cross-validation with the best model from GridSearchCV
cv_scores_grid = cross_val_score(clf_grid.best_estimator_, X_new, y, cv=10)
cv_mean_accuracy_grid = cv_scores_grid.mean()

accuracy_grid, cv_mean_accuracy_grid


In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

# Separating features and target variable
X = data.drop(columns='class')
y = data['class']

# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the data (important for SGD and MLP)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 1. Train SGDClassifier
sgd_clf = SGDClassifier(random_state=42)
sgd_clf.fit(X_train_scaled, y_train)

# Accuracy for train-test split
y_pred_sgd = sgd_clf.predict(X_test_scaled)
accuracy_sgd_split = accuracy_score(y_test, y_pred_sgd)

# Cross-validation accuracy
sgd_cv_scores = cross_val_score(sgd_clf, X_train_scaled, y_train, cv=5)
accuracy_sgd_cv = sgd_cv_scores.mean()

# 2. Train RandomForestClassifier
rf_clf = RandomForestClassifier(random_state=42)
rf_clf.fit(X_train, y_train)

# Accuracy for train-test split
y_pred_rf = rf_clf.predict(X_test)
accuracy_rf_split = accuracy_score(y_test, y_pred_rf)

# Cross-validation accuracy
rf_cv_scores = cross_val_score(rf_clf, X_train, y_train, cv=5)
accuracy_rf_cv = rf_cv_scores.mean()

# 3. Train MLPClassifier
mlp_clf = MLPClassifier(random_state=42, max_iter=300)
mlp_clf.fit(X_train_scaled, y_train)

# Accuracy for train-test split
y_pred_mlp = mlp_clf.predict(X_test_scaled)
accuracy_mlp_split = accuracy_score(y_test, y_pred_mlp)

# Cross-validation accuracy
mlp_cv_scores = cross_val_score(mlp_clf, X_train_scaled, y_train, cv=5)
accuracy_mlp_cv = mlp_cv_scores.mean()

# Output the results
accuracy_sgd_split, accuracy_sgd_cv, accuracy_rf_split, accuracy_rf_cv, accuracy_mlp_split, accuracy_mlp_cv
